# Setup

In [1]:
import os
from typing import Tuple

In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from PIL import Image

/home/kieefu/apps/mini-env/envs/eda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Setup variables

In [3]:
DEVICE: str = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
MODEL_PATH: str = 'artifacts/correted_21_0102_best.pth'
IMAGE_FOLDER: str = 'artifacts/test'
SUBMISSION_FILE: str = 'artifacts/test-labels.csv'
SUBMISSION_RESULT: str = 'artifacts/test-submission.csv'

CLASS_NAMES: Tuple = ('2', '3', '4', '5', '6', '7', '8', 
                    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 
                    'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 
                    'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 
                    'y', 'z')

In [4]:
class ClassiNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(2304, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 33)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch 4x2304
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
def predict(model, image) -> Tuple[float, int, torch.Tensor]:
    with torch.no_grad():
        output = torch.nn.functional.softmax(model(image), dim=1)
        probability, prediction = torch.max(output, 1)
    return probability.item(), prediction.item(), output


In [6]:
data_transforms = {'test': transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5))])
    }

In [7]:
def load_inference_data(image_path) -> torch.Tensor:
    orig_image = Image.open(image_path)
    image = data_transforms['test'](orig_image)
    image = image.unsqueeze(0)
    return image, orig_image

In [8]:
classi_net = ClassiNet()
classi_net.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
classi_net.eval()

ClassiNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=2304, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=33, bias=True)
)

In [9]:
dataset_df = pd.read_csv(SUBMISSION_FILE)
dataset_df.head()

,id,img,label
0,0,uqxwnf.png,0
1,1,uaevcj.png,0
2,2,gxtrws.png,0
3,3,tcdqdh.png,0
4,4,uprnyk.png,0


In [10]:
predictions = []

for index, value in dataset_df.iterrows():
    image, orig_image = load_inference_data(os.path.join(IMAGE_FOLDER, value['img']))
    probability, prediction, output = predict(classi_net, image)

    predictions.append(CLASS_NAMES[prediction])

In [12]:
result_df = pd.DataFrame()
result_df['label'] = predictions
result_df.head()

,label
0,o
1,3
2,2
3,3
4,5


In [ ]:
result_df.to_csv(SUBMISSION_RESULT)